In [3]:
import pandas as pd
pd.set_option("display.max_row", 1000)
pd.set_option("display.max_column", 100)
import warnings
warnings.filterwarnings('ignore')

path = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/"

**7회-1번) 결측치 처리 및 과목별 점수 표준화**

학생들의 수업 과목별 점수 데이터를 변환하고자 한다.

다음 절차에 따라 문제를 해결하시오.
- 1) 'score' 컬럼에서 결측치가 발생한 행을 제거한 뒤, 가장 많은 학생이 수강한 과목(subject)을 찾으시오.
- 2) 해당 과목의 점수(score)를 표준화(standardization)하여, 표준화된 점수 중 가장 높은 값을 구하시오.
> 표준화 공식: (값 - 평균) / 표준편차
- 3) 구한 값을 반올림하여 소수점 아래 3자리까지 출력한다.

In [24]:
df = pd.read_csv(path + "student_scores01.csv")
# print(df.head(), df.info(), sep="\n")

# 'score' 컬럼에서 결측치가 발생한 행을 제거한 뒤, 가장 많은 학생이 수강한 과목(subject)을 찾으시오.
df = df[~df['score'].isnull()].copy()
# print(df.head())
max_subject = df.groupby('subject')['subject'].count().reset_index(name='count').max()[0]
# print(max_subject)

# 해당 과목의 점수(score)를 표준화(standardization)하여, 표준화된 점수 중 가장 높은 값을 구하시오.
# 표준화 공식: (값 - 평균) / 표준편차
df2 = df.loc[df['subject'] == max_subject].copy()
result = round(((df['score'] - df['score'].mean()) / df['score'].std(ddof=1)).max(), 3)
# print(result) # 1.663


1.663


7회-2번) 종속변수와의 상관관계 분석

주어진 데이터에서 종속변수와 가장 높은 상관관계를 갖는 독립변수를 찾고자 한다.

다음 절차에 따라 문제를 해결하시오.

* 1. 해당 데이터에는 77개의 독립변수와 1개의 종속변수('CLOSE')가 존재한다.
* 2. 종속변수 'CLOSE'와 가장 높은 상관관계를 갖는 독립변수를 찾는다.
* 3. 찾은 독립변수의 평균값을 구하고, 반올림하여 소수점 아래 3자리까지 출력한다.
- [주의] 문제에서 상관관계에 대한 방법을 명시하지 않으면 pearson 방법입니다.

In [68]:
df = pd.read_csv(path + "close_features.csv")
# print(df.head(), df.info(), sep="\n")
# print(df.isna().sum().to_frame())

# 종속변수 'CLOSE'와 가장 높은 상관관계를 갖는 독립변수를 찾는다.
df = df.dropna()
corr_df = df.corr(method="pearson")['CLOSE']
corr_df = corr_df.drop("CLOSE")
best_var = corr_df.abs().idxmax() 
# print(best_var) # feature_31

# 찾은 독립변수의 평균값을 구하고, 반올림하여 소수점 아래 3자리까지 출력한다.
result = round(df[best_var].mean(), 3)
# print(result) # 4.807


4.807


7회-3번) 이상치 개수 구하기

주어진 데이터에서 특정 기준에 따라 이상치 개수를 구하고자 한다.

다음 절차에 따라 문제를 해결하시오.

* 'feature_2' 컬럼의 이상치 개수를 구하여 정수로 출력한다.
* 이상치 기준은 다음과 같다.
- - 이상치 < 평균 - IQR * 1.5, 이상치 > 평균 + IQR * 1.5

In [103]:
df = pd.read_csv(path + "outlier_data.csv")
# print(df.head())
mean = df['feature_2'].mean()
q1, q3 = df['feature_2'].quantile([0.25, 0.75])
# print(q1, q3) # -71.0 75.0
iqr = q3 - q1

cond1 = mean - (iqr * 1.5)
cond2 = mean + (iqr * 1.5)

feature_2 = len(df[(df['feature_2'] < cond1) | (df['feature_2'] > cond2)]['feature_2'])
# print(feature_2) # 224


224


15-4) 이상치가 가장 많은 독립변수 찾기 및 이상치 개수 구하기

주어진 데이터는 아파트 가격 관련 데이터를 포함하고 있다. 아파트 가격(Price)은 종속변수이며, 나머지 변수들은 독립변수이다.

다음 절차에 따라 문제를 해결하시오.

* 1. 아파트 가격을 결정하는 독립변수들을 대상으로 분석을 수행한다.
* 2. 이상치 기준은 다음과 같다.
- -  이상치 < 평균 - IQR * 1.5, 이상치 > 평균 + IQR * 1.5

* 3. 모든 독립변수에 대해 이상치 개수를 구한다.
* 4. 이상치 개수가 가장 많은 독립변수를 찾아, 이상치 개수를 정수로 출력합니다.

In [113]:
df = pd.read_csv(path + "apartment_prices.csv")
# print(df.head())
# print(df.isna().sum().to_frame().T)
# print(df.columns)

# 독립변수 리스트
cols = ['Size', 'Bedrooms', 'Bathrooms', 'Floor', 'Age', 'DistanceToCenter',
        'NearbySchools', 'NearbyParks', 'PublicTransport', 'SafetyIndex']

# 독립변수별 이상치 개수 구하기
values = []
for col in cols:
    mean = df[col].mean()
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    cond1 = mean - (iqr * 1.5)
    cond2 = mean + (iqr * 1.5)
    cnt = len(df[(df[col] < cond1) | (df[col] > cond2)])
    values.append([col, cnt])
df2 = pd.DataFrame(values, columns=['col','cnt'])

# 이상치 개수가 가장 많은 독립변수를 찾아, 이상치 개수를 정수로 출력
result = int(df2.loc[df2.index == df2['cnt'].idxmax(), 'cnt'])
# print(result) # 16


16


15-5) 상관관계 분석

주어진 데이터에서 독립변수들 간의 상관관계를 분석하고자 한다.

다음 절차에 따라 문제를 해결하시오.

* 1. 결측치가 포함된 모든 행을 제거합니다.
* 2. 1)의 결과를 사용해 모든 숫자형 변수 간 상관계수를 'spearman'으로 계산하시오.
* 3. 숫자형 변수들 중 상관관계가 가장 큰 두 변수를 찾으시오.
* 단, 같은 변수 간의 상관계수(자기 자신과의 상관계수)는 제외한다.
* 4. 해당 두 변수의 상관계수 값을 반올림하여 소수점 아래 2자리까지 출력한다.
- [참고] 'pearson', 'spearman', 'kendall' 모두 해보세요.

* * 일반적으로는 'pearson' 이지만 차이가 있는지 보는 것도 재미있죠!

In [173]:
df = pd.read_csv(path + "penguins01.csv")
# print(df.head(3))
# print(df.isna().sum().to_frame().T)

# 결측치가 포함된 모든 행을 제거
df = df.dropna()
# print(df.isna().sum().to_frame().T)

# 1)의 결과를 사용해 모든 숫자형 변수 간 상관계수를 'spearman'으로 계산
# print(df.info())
df2 = df.select_dtypes(include=['number']).copy()
corr = df2.corr(method='spearman') # 0.84
# corr = df2.corr(method='pearson')  # 0.87
# corr = df2.corr(method='kendall')    # 0.66
# print(corr)

# 숫자형 변수들 중 상관관계가 가장 큰 두 변수를 찾으시오. 자기자신과의 상관계수는 제외
corr_pair = corr.unstack().reset_index()
corr_pair.columns = ['var1', 'var2', 'correlation']
corr_pair = corr_pair[corr_pair['var1'] != corr_pair['var2']]
vars = corr_pair.loc[corr_pair.index==corr_pair['correlation'].idxmax(), ['var1','var2']].values[0]
# print(vars) # ['flipper_length_mm' 'body_mass_g']

# 해당 두 변수의 상관계수 값을 반올림하여 소수점 아래 2자리까지 출력한다.
result = round(corr_pair[corr_pair['var1'].isin(vars)]['correlation'].max(), 2)
# print(result) # 0.84


0.84


15-6) 상관관계 분석 2

주어진 데이터에서 독립변수들 간의 상관관계를 분석하고자 한다.

다음 절차에 따라 문제를 해결하시오.

* 1. 모든 "숫자형 변수" 간 상관계수(Pearson correlation coefficient)를 계산하시오.
* 단, 같은 변수 간의 상관계수(자기 자신과의 상관계수)는 제외한다.

* 2. 상관계수의 절댓값이 0.8 이상인 변수쌍의 개수를 구하여 정수로 출력하시오.

In [181]:
df = pd.read_csv(path + "mpg.csv")
# print(df.head(3))
# print(df.isna().sum().to_frame().T)

# 모든 "숫자형 변수" 간 상관계수(Pearson correlation coefficient)를 계산하시오.
# 단, 같은 변수 간의 상관계수(자기 자신과의 상관계수)는 제외한다.
df2 = df.select_dtypes(include=['number']).copy()
corr2 = df2.corr(method="pearson")
corr2 = corr2.unstack().reset_index()
corr2.columns = ['var1', 'var2', 'value']
corr2 = corr2[corr2['var1'] != corr2['var2']].copy()
# print(corr2)
# 상관계수의 절댓값이 0.8 이상인 변수쌍의 개수를 구하여 정수로 출력하시오.
result = corr2[abs(corr2['value']) >= 0.8]['var1'].count() * 2
# print(result) # 32


32
